# Session 3: Data Cleaning and Feature Engineering
## When Models Break and How to Fix the Input

**Session Length:** 2 hours

**Today's Mission:** Discover what makes AI models fail, learn a framework for getting AI to write code for you, and use AI-assisted coding to fix broken inputs.

### Session Outline
| Time | Activity |
|------|----------|
| 0:00-0:05 | Review: What pipelines did you explore? |
| 0:05-0:35 | Part 1: Breaking Models on Purpose |
| 0:35-1:05 | Part 2: AI-Assisted Coding (CLEAR Framework) |
| 1:05-1:40 | Part 3: Fix the Input |
| 1:40-2:00 | On Your Own: Code reading + modification challenges |

### Key Vocabulary

| Term | Definition |
|------|-----------|
| **Data Quality** | How clean, complete, and reliable the input data is |
| **Noise** | Random errors or irrelevant information in data |
| **Adversarial Input** | Data deliberately crafted to confuse a model |
| **Domain Mismatch** | When input data is different from training data |
| **CLEAR Framework** | Context, Language, Explain, Ask, Requirements -- for prompting AI |

---

## Review (0:00-0:05)

Last session we explored how data gets represented as numbers so that computers can work with it. We saw that representation choices matter -- the same data in different formats enables different computations.

Today we flip the question: **What happens when the input data is messy, weird, or deliberately confusing?**

Every AI model you have used so far was tested on clean, well-formed text. Real-world inputs are rarely that tidy. Let's find out what happens when things go wrong.

---

## Part 1: Breaking Models on Purpose (0:05-0:35)

In Sessions 1 and 2, we focused on making models work correctly. Today we do the opposite: **we are going to try to break them.**

Why? Because understanding failure is how real AI researchers improve systems. Every limitation you discover has a name in machine learning, and knowing those names is the first step toward fixing problems.

Let's start by loading a sentiment analysis model and feeding it increasingly difficult inputs.

In [ ]:
# Install and import
!pip install transformers==4.47.1 gradio -q

from transformers import pipeline
print("Ready!")

**After running the install cell above**, go to **Runtime > Restart runtime**, then continue from the cell below. This ensures all packages load correctly. This is standard practice -- every data scientist does this.

> **FIND A MODEL: Text Classification**
>
> Before we start breaking models, let's see what's available. The default sentiment model was trained on English movie reviews -- but there are hundreds of text-classification models on Hugging Face, trained on tweets, product reviews, news, toxic comments, and more.
>
> 1. Go to [huggingface.co/models?pipeline_tag=text-classification&sort=downloads](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads)
> 2. Browse the top results. Notice the variety -- sentiment, emotion, spam, toxicity, topic classification.
> 3. Pick one model and **read its model card**: What was it trained on? What language? Any known limitations?
>
> Keep your model in mind -- later in this session you'll build a Gradio app and can swap in any model you found.

In [ ]:
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment = pipeline("sentiment-analysis")
print("Sentiment model loaded!")

### Step 1: Clean Input -- Everything Works

Let's start with a perfectly clear, well-written sentence. This is what the model was designed to handle.

In [ ]:
# Clean, clear text -- the model's comfort zone
clean_text = "I absolutely loved this movie. The acting was brilliant and the story was captivating."
result = sentiment(clean_text)[0]
print(f"Text: {clean_text}")
print(f"Result: {result['label']} (confidence: {result['score']:.1%})")

That worked perfectly. High confidence, correct answer. Now let's see what happens when we start making the input worse.

### Step 2: Typos and Misspellings -- Noise

In [ ]:
# Typos and misspellings
noisy_text = "Ths moive ws realy gret!"
result = sentiment(noisy_text)[0]
print(f"Text: {noisy_text}")
print(f"Result: {result['label']} (confidence: {result['score']:.1%})")
print()
print("Compare to the clean version:")
clean_result = sentiment("This movie was really great!")[0]
print(f"Text: This movie was really great!")
print(f"Result: {clean_result['label']} (confidence: {clean_result['score']:.1%})")
print()
print("ML term: This is NOISE -- random errors that degrade the signal.")

### Step 3: Sarcasm -- Ambiguity

In [ ]:
# Sarcasm -- says one thing, means the opposite
sarcastic_texts = [
    "Oh great, another Monday. Just what I needed.",
    "Wow, what a surprise. The bus is late again.",
    "Sure, I love waiting in line for two hours. Best day ever.",
]

print("Sarcasm test:")
print("=" * 60)
for text in sarcastic_texts:
    result = sentiment(text)[0]
    print(f"Text: {text}")
    print(f"Model says: {result['label']} ({result['score']:.1%})")
    print()

print("ML term: This is AMBIGUITY -- the literal words")
print("conflict with the intended meaning.")

### Step 4: Mixed Languages -- Domain Mismatch

In [ ]:
# Mixed languages -- the model was trained on English
mixed_texts = [
    "This movie was tres magnifique but also schlecht",
    "The food was oishii and the service was terrible",
    "Me encanta this place pero the prices are muy altos",
]

print("Mixed language test:")
print("=" * 60)
for text in mixed_texts:
    result = sentiment(text)[0]
    print(f"Text: {text}")
    print(f"Model says: {result['label']} ({result['score']:.1%})")
    print()

print("ML term: This is DOMAIN MISMATCH -- the model was trained")
print("on English text, so other languages confuse it.")

### Step 4.5: Spam vs Ham -- Another Classification Task (Book Enhancement)

Spam detection is another text-classification problem. Real systems use this for email, comments, and DMs.

We will test normal spam/ham messages first, then adversarial versions where spam is disguised.

In [ ]:
# Spam classification model from the book
spam_classifier = pipeline(
    "text-classification",
    model="Delphia/twitter-spam-classifier"
)

messages = [
    "Congratulations! You won a free prize. Click this link now.",
    "Hi team, reminder: project check-in starts at 3:30 PM.",
    "URGENT: Your account will be closed unless you verify today.",
    "Can you send me the notes from math class?"
]

print("SPAM VS HAM")
print("=" * 60)
for msg in messages:
    result = spam_classifier(msg)[0]
    label = "SPAM" if str(result['label']) == "1" else "HAM"
    print(f"Message: {msg}")
    print(f"Prediction: {label} ({result['score']:.1%})")
    print()

In [ ]:
# Adversarial spam: same intent, obfuscated wording
adversarial_messages = [
    "C0NGRATS!!! cl1ck n0w to claim your g1ft card",
    "Fr33 prize waiting, tap here ASAP",
    "Hey, can we meet after school to review chapter 4?"
]

print("ADVERSARIAL SPAM TEST")
print("=" * 60)
for msg in adversarial_messages:
    result = spam_classifier(msg)[0]
    label = "SPAM" if str(result['label']) == "1" else "HAM"
    print(f"Message: {msg}")
    print(f"Prediction: {label} ({result['score']:.1%})")
    print()

> **READ THE MODEL CARD**
>
> We just used `Delphia/twitter-spam-classifier` — but should we trust it? Check the source:
>
> Go to [huggingface.co/Delphia/twitter-spam-classifier](https://huggingface.co/Delphia/twitter-spam-classifier)
>
> - What data was it trained on?
> - How many examples did it learn from?
> - Is it designed for tweets, emails, or something else?
> - What limitations does it mention?
>
> Reading model cards is how professionals decide whether to trust a model for their use case. A spam detector trained on Twitter may not catch email spam — that's domain mismatch.

> **INSTRUCTOR NOTE:** Ask students to invent one spam message and one normal message. Test both and compare confidence.

In [ ]:
# Student spam test
student_message = "REPLACE WITH STUDENT SUGGESTION"

if "REPLACE" not in student_message:
    result = spam_classifier(student_message)[0]
    label = "SPAM" if str(result['label']) == "1" else "HAM"
    print(f"Message: {student_message}")
    print(f"Prediction: {label} ({result['score']:.1%})")

### Step 5: Extreme Length -- Short vs Long

In [ ]:
# Very short input
short_text = "ok"
result = sentiment(short_text)[0]
print(f"Short text: '{short_text}'")
print(f"Model says: {result['label']} ({result['score']:.1%})")
print()

# Very long, repetitive input
long_text = "This was good. " * 50
result = sentiment(long_text[:512])[0]  # Models have max length
print(f"Long text: '{long_text[:60]}...' (repeated 50 times)")
print(f"Model says: {result['label']} ({result['score']:.1%})")
print()
print("Short inputs give the model very little to work with.")
print("Long inputs may get truncated -- the model only sees the first part.")

### Step 6: Wrong Task -- Adversarial Input

Let's try a completely different kind of attack: asking the model to analyze text that has nothing to do with sentiment.

In [ ]:
# Text with no sentiment at all
neutral_texts = [
    "The chemical formula for water is H2O.",
    "Paris is the capital of France.",
    "There are 24 hours in a day.",
]

print("Neutral/factual text test:")
print("=" * 60)
for text in neutral_texts:
    result = sentiment(text)[0]
    print(f"Text: {text}")
    print(f"Model says: {result['label']} ({result['score']:.1%})")
    print()

print("These are factual statements with no sentiment.")
print("But the model MUST pick positive or negative -- it has no 'neutral' option.")
print("This is a design limitation, not a data problem.")

### Step 7: Question Answering Under Pressure

Sentiment is not the only model we can break. Let's try a question-answering model with a question that is NOT in the context.

In [ ]:
# Load QA pipeline
qa = pipeline("question-answering")

# Ask something that IS in the context
context = "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize."
question_good = "What did Marie Curie research?"
result = qa(question=question_good, context=context)
print("Good question (answer IS in the context):")
print(f"  Q: {question_good}")
print(f"  A: {result['answer']} (confidence: {result['score']:.1%})")
print()

# Ask something NOT in the context
question_bad = "What is Marie Curie's favorite food?"
result = qa(question=question_bad, context=context)
print("Bad question (answer is NOT in the context):")
print(f"  Q: {question_bad}")
print(f"  A: {result['answer']} (confidence: {result['score']:.1%})")
print()
print("The model tries to answer even when the answer is not there.")
print("Notice the low confidence -- that is the model saying 'I am guessing.'")

### Step 8: Zero-Shot with Ambiguous Categories

Let's try one more model: zero-shot classification with categories that all kind of fit.

In [ ]:
# Load zero-shot classifier
classifier = pipeline("zero-shot-classification")

# Ambiguous text with overlapping categories
ambiguous_text = "I spent the weekend painting landscapes in the park while listening to music"
categories = ["art", "nature", "music", "exercise", "relaxation"]

result = classifier(ambiguous_text, categories)
print(f"Text: {ambiguous_text}")
print(f"\nCategories and scores:")
for label, score in zip(result['labels'], result['scores']):
    bar = "*" * int(score * 30)
    print(f"  {label:12} {bar} ({score:.1%})")
print()
print("When multiple categories fit, the model spreads its confidence.")
print("This is not a failure -- it is honest uncertainty.")

> **INSTRUCTOR NOTE:** Ask students: "What input do you think would confuse this model? What sentence could you write that would be hard for ANY model to classify?" Let them suggest adversarial inputs. Type them in the cell below and run it live.

In [ ]:
# Student adversarial input -- try to break the model!
student_breaker = "REPLACE WITH STUDENT SUGGESTION -- TRY TO BREAK THE MODEL"

# Try it on sentiment
result_sent = sentiment(student_breaker)[0]
print(f"Sentiment says: {result_sent['label']} ({result_sent['score']:.1%})")

# Try it on zero-shot
result_zs = classifier(student_breaker, ["positive", "negative", "confusing", "nonsense"])
print(f"\nZero-shot says: {result_zs['labels'][0]} ({result_zs['scores'][0]:.1%})")

### Summary: Why Models Break

Every one of these problems has a name in machine learning:

| Problem | ML Term | Example |
|---------|---------|---------|
| Typos and errors | **Noise** | "Ths moive ws gret" |
| Sarcasm | **Ambiguity** | "Oh great, another Monday" |
| Foreign words | **Domain mismatch** | "tres magnifique" |
| Very short text | **Insufficient signal** | "ok" |
| No right answer exists | **Design limitation** | Sentiment on factual text |
| Deliberately confusing | **Adversarial input** | Student suggestions! |

Real AI systems need to handle all of this. The first step is recognizing the problem -- and now you can.

> **INSTRUCTOR NOTE:** "Show a model card's 'Limitations' section on Hugging Face. Go to huggingface.co/distilbert-base-uncased-finetuned-sst-2-english and scroll to 'Limitations and bias'. Point out: Every model tells you what it's bad at. Reading this section is a professional skill."

---

## Part 2: AI-Assisted Coding -- The CLEAR Framework (0:35-1:05)

Now that we know what breaks models, let's learn a skill that real programmers use every day: **getting AI to write code for you.**

You do not need to memorize Python syntax. You need to:
1. **Know what you want to build**
2. **Ask AI to write the code clearly**
3. **Understand what the AI wrote**
4. **Modify it to do what you want**

The key to step 2 is a good prompt. Bad prompts give bad code. Good prompts give code you can actually use.

### The CLEAR Framework

| Letter | Meaning | Example |
|--------|---------|---------|
| **C** | Context | "I'm working in Google Colab with Python" |
| **L** | Language/Libraries | "Using the transformers library" |
| **E** | Explain the goal | "I want to analyze homework assignments" |
| **A** | Ask specifically | "Write a function called homework_analyzer" |
| **R** | Requirements | "Include comments explaining each step" |

### Bad Prompt vs Good Prompt

**Bad prompt:**
> "Write code that analyzes homework"

**Good CLEAR prompt:**
> "I'm working in Google Colab with Python. Using the Hugging Face transformers library, write a function called `homework_analyzer` that:
> - Takes a string of text (a homework answer) as input
> - Uses a summarization pipeline to create a 1-sentence summary
> - Uses a sentiment pipeline to assess the tone (confident vs uncertain)
> - Returns a dictionary with the summary, sentiment label, and confidence score
> - Includes comments explaining each step
> - Prints a formatted report at the end"

Which prompt do you think will give better results?

> **INSTRUCTOR NOTE:** "Open Claude or ChatGPT in a new tab. Show students the CLEAR prompt above. Type it live -- or paste it. Show the response. Copy the generated code into Colab. Run it. Debug together if needed. This entire Part 2 IS the AI demo moment."

### The Live Exercise

The cell below contains a pre-written version of what the AI should generate. If the live demo works, use the AI's code instead. If it has bugs, use this as a fallback.

In [ ]:
from transformers import pipeline

def homework_analyzer(text):
    """Analyze a homework answer: summarize it and assess the tone."""

    # Step 1: Load the summarization pipeline
    summarizer = pipeline("summarization")

    # Step 2: Load the sentiment pipeline
    sentiment_checker = pipeline("sentiment-analysis")

    # Step 3: Summarize the homework answer
    # min_length and max_length control the summary size
    if len(text.split()) > 30:
        summary = summarizer(text, max_length=50, min_length=10, do_sample=False)
        summary_text = summary[0]['summary_text']
    else:
        summary_text = text  # Too short to summarize

    # Step 4: Check the sentiment (confident vs uncertain tone)
    tone = sentiment_checker(text)[0]

    # Step 5: Build the result dictionary
    result = {
        'summary': summary_text,
        'tone': tone['label'],
        'confidence': round(tone['score'], 3)
    }

    # Step 6: Print a formatted report
    print("=" * 50)
    print("HOMEWORK ANALYSIS REPORT")
    print("=" * 50)
    print(f"Original ({len(text.split())} words):")
    print(f"  {text[:100]}{'...' if len(text) > 100 else ''}")
    print(f"\nSummary:")
    print(f"  {summary_text}")
    print(f"\nTone: {tone['label']} ({tone['score']:.1%} confidence)")
    print("=" * 50)

    return result

print("homework_analyzer function is ready!")

In [ ]:
# Test the homework analyzer
sample_homework = """The water cycle begins when the sun heats water in oceans,
lakes, and rivers, causing it to evaporate into the atmosphere. The water vapor
rises and cools, forming clouds through condensation. When the clouds become
heavy enough, precipitation falls as rain, snow, or hail. The water then flows
back into bodies of water through runoff and the cycle begins again. This
continuous process is essential for distributing fresh water around the planet."""

result = homework_analyzer(sample_homework)

> **ASK AI ABOUT THIS**
>
> Copy the `homework_analyzer` function into Claude or ChatGPT and ask:
> *"Can you explain what each line does in plain English?"*
>
> This is how real programmers learn -- by asking questions about code they encounter.

### Your Turn: Try Different Inputs

Test the analyzer with different kinds of homework answers. What happens with short answers? Confident answers? Uncertain ones?

In [ ]:
# REPLACE WITH STUDENT SUGGESTION -- try different homework answers
student_homework = "REPLACE WITH STUDENT SUGGESTION"

result = homework_analyzer(student_homework)

---

## Part 3: Fix the Input (1:05-1:25)

Now let's connect Parts 1 and 2. In Part 1, we discovered what breaks models. In Part 2, we learned to use AI to write code. Now let's **use AI-assisted coding to fix the broken inputs from Part 1.**

### A Simple Text Cleaning Function

The function below handles some of the most common input problems: extra whitespace, inconsistent capitalization, and repeated characters.

In [ ]:
import re

def clean_text(text):
    """Clean up messy text before sending it to a model."""
    # Remove extra whitespace
    text = ' '.join(text.split())

    # Fix repeated characters (e.g., "sooooo" -> "soo")
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Fix common typos/abbreviations
    replacements = {
        'u ': 'you ',
        'ur ': 'your ',
        'r ': 'are ',
        'thx': 'thanks',
        'pls': 'please',
        'bc ': 'because ',
        'w/': 'with ',
        'w/o': 'without',
    }
    for old, new in replacements.items():
        text = text.replace(old, new)

    return text.strip()

# Demo the cleaning
messy_examples = [
    "   this   movie    was   soooooo    good   ",
    "u should watch this bc its amazinggggg",
    "thx for the recommendation pls send more",
]

print("Text cleaning demo:")
print("=" * 60)
for messy in messy_examples:
    cleaned = clean_text(messy)
    print(f"Before: '{messy}'")
    print(f"After:  '{cleaned}'")
    print()

### Re-Running Broken Inputs Through Cleaning

Let's take some of the inputs that confused the model in Part 1 and see if cleaning helps.

In [ ]:
from transformers import pipeline
sentiment = pipeline("sentiment-analysis")

# The broken inputs from Part 1
broken_inputs = [
    "Ths moive ws realy gret!",
    "   this   movie    was   soooooo    good   ",
    "u should watch this bc its amazinggggg",
]

print("Before and after cleaning:")
print("=" * 70)
for text in broken_inputs:
    # Analyze the messy version
    messy_result = sentiment(text)[0]

    # Clean it, then analyze
    cleaned = clean_text(text)
    clean_result = sentiment(cleaned)[0]

    print(f"Messy:   '{text}'")
    print(f"  Model: {messy_result['label']} ({messy_result['score']:.1%})")
    print(f"Cleaned: '{cleaned}'")
    print(f"  Model: {clean_result['label']} ({clean_result['score']:.1%})")
    change = clean_result['score'] - messy_result['score']
    if abs(change) > 0.001:
        direction = "higher" if change > 0 else "lower"
        print(f"  Confidence change: {abs(change):.1%} {direction}")
    print()

### What Cleaning Cannot Fix

Some problems are not about messy text -- they are about **meaning**. Let's test the limits of cleaning.

In [ ]:
# Sarcasm -- cleaning does not help here
sarcastic = "Oh great, another Monday. Just what I needed."
cleaned_sarcastic = clean_text(sarcastic)

result_before = sentiment(sarcastic)[0]
result_after = sentiment(cleaned_sarcastic)[0]

print("Sarcasm test:")
print(f"  Original: '{sarcastic}'")
print(f"  Cleaned:  '{cleaned_sarcastic}'")
print(f"  Before cleaning: {result_before['label']} ({result_before['score']:.1%})")
print(f"  After cleaning:  {result_after['label']} ({result_after['score']:.1%})")
print()
print("Cleaning did not help. The text was already clean.")
print("Sarcasm is a MEANING problem, not a FORMATTING problem.")
print("Fixing sarcasm requires a model that understands context and tone --")
print("that is a much harder problem than text cleaning.")

> **ASK AI ABOUT THIS**
>
> Copy the `clean_text` function into Claude or ChatGPT and ask:
> *"What other text cleaning steps would help a sentiment model? Can you add 3 more improvements?"*
>
> This is how real programmers learn -- by asking questions about code they encounter.

### Discussion

**What can cleaning fix?**
- Extra whitespace, repeated characters, common abbreviations
- Inconsistent formatting

**What can cleaning NOT fix?**
- Sarcasm and irony (meaning problems)
- Domain mismatch (wrong language)
- Missing information (too short to analyze)
- Design limitations (no neutral option)

This distinction -- **formatting problems vs meaning problems** -- is important. Data scientists spend most of their time cleaning data, but they also need to know when cleaning is not enough.

---

### Bonus: Share Your Model as an App

You have been running models in code cells. But what if you wanted to share a sentiment analyzer with someone who does not know Python?

**Gradio** turns any Python function into a web app in a few lines of code. When you run the cell below, it creates a link anyone can click -- no coding required on their end.

**But first -- which model powers your app?** You can use the default sentiment model, or swap in one you found on the Hub.

In [ ]:
# ── SWAP SLOT: Choose the model for your Gradio app ──
# Option A: Use the default sentiment model (already loaded above)
# Option B: Swap in a model you found on the Hub!

my_model_id = "PASTE YOUR MODEL ID HERE"
# Example: "cardiffnlp/twitter-roberta-base-sentiment-latest"
# Example: "finiteautomata/bertweet-base-sentiment-analysis"

# Uncomment these two lines to use your own model:
# from transformers import pipeline
# sentiment = pipeline("text-classification", model=my_model_id)

# Then run the Gradio cell below -- it will use whichever model
# is loaded as `sentiment`
print("Using default sentiment model. Uncomment above to swap!")

In [ ]:
import gradio as gr

def check_sentiment(text):
    result = sentiment(text)[0]
    return f"{result['label']} ({result['score']:.1%})"

demo = gr.Interface(
    fn=check_sentiment,
    inputs=gr.Textbox(label="Type any text", lines=2,
                     placeholder="Try sarcasm, slang, or mixed feelings..."),
    outputs=gr.Textbox(label="Sentiment"),
    title="Sentiment Checker",
    allow_flagging="never",
)

demo.launch(share=True)

> **INSTRUCTOR NOTE:** The `share=True` flag creates a public URL that works for about 72 hours. Students can text this link to friends or family and let them try the sentiment checker. This is their first "I built an AI app" moment. Stop the demo by clicking the stop button or restarting the runtime when done.

---

## On Your Own (1:40-2:00)

### Exercise 1: Code Detective

Look at this code and answer the questions WITHOUT running it:

In [ ]:
# DON'T RUN THIS YET -- just read it!

from transformers import pipeline

def mystery_function(texts):
    classifier = pipeline("zero-shot-classification")
    categories = ["urgent", "normal", "spam"]

    results = []
    for text in texts:
        result = classifier(text, categories)
        top_category = result['labels'][0]
        confidence = result['scores'][0]
        results.append({
            'message': text,
            'category': top_category,
            'confidence': confidence
        })

    urgent_count = sum(1 for r in results if r['category'] == 'urgent')

    return results, urgent_count

**Questions (answer before running):**

1. What does this function do?
   - Your answer:

2. What type of input does it expect?
   - Your answer:

3. What does it return?
   - Your answer:

4. What would be a good name for this function instead of `mystery_function`?
   - Your answer:

5. What would happen if you gave it an empty list?
   - Your answer:

**Now run it to check your answers:**

In [ ]:
# Now run and test it
from transformers import pipeline

def mystery_function(texts):
    classifier = pipeline("zero-shot-classification")
    categories = ["urgent", "normal", "spam"]

    results = []
    for text in texts:
        result = classifier(text, categories)
        top_category = result['labels'][0]
        confidence = result['scores'][0]
        results.append({
            'message': text,
            'category': top_category,
            'confidence': confidence
        })

    urgent_count = sum(1 for r in results if r['category'] == 'urgent')

    return results, urgent_count

# Test it
test_messages = [
    "URGENT: Your account will be suspended!",
    "Meeting tomorrow at 3pm.",
    "You've won a FREE iPhone! Click here!",
    "Please review the attached document when you have time."
]

results, urgent = mystery_function(test_messages)
print(f"Found {urgent} urgent messages\n")
for r in results:
    print(f"{r['category'].upper()}: {r['message'][:50]}...")

### Exercise 2: Modify the Code

Can you modify the code above to:
1. Add a new category: "promotional"
2. Also count spam messages
3. Print a warning if more than half the messages are urgent or spam

Try to do this yourself first, then use AI if you get stuck:

In [ ]:
# YOUR MODIFIED VERSION
# Try to make the changes yourself!

from transformers import pipeline

def email_sorter(texts):
    classifier = pipeline("zero-shot-classification")
    # TODO: Add "promotional" to categories
    categories = ["urgent", "normal", "spam"]

    results = []
    for text in texts:
        result = classifier(text, categories)
        top_category = result['labels'][0]
        confidence = result['scores'][0]
        results.append({
            'message': text,
            'category': top_category,
            'confidence': confidence
        })

    urgent_count = sum(1 for r in results if r['category'] == 'urgent')
    # TODO: Count spam too

    # TODO: Add warning if more than half are urgent or spam

    return results, urgent_count

# Test your modified version
test_messages = [
    "URGENT: Your account will be suspended!",
    "Meeting tomorrow at 3pm.",
    "You've won a FREE iPhone! Click here!",
    "50% off all items this weekend only!",
    "Please review the attached document when you have time."
]

results, urgent = email_sorter(test_messages)
for r in results:
    print(f"{r['category'].upper()}: {r['message'][:40]}...")

### Exercise 3: Modify the Homework Analyzer

Add sentence counting to `homework_analyzer`: report whether the answer is "too short" (1-2 sentences), "good length" (3-5 sentences), or "detailed" (6+ sentences).

**Hint:** `text.count('.') + text.count('!') + text.count('?')` gives you sentence count.

In [ ]:
# YOUR MODIFIED homework_analyzer
# Add sentence counting and length assessment

# Hint: text.count('.') + text.count('!') + text.count('?') gives you sentence count

> **ASK AI ABOUT THIS**
>
> If you get stuck on Exercise 3, copy the original `homework_analyzer` function into Claude and ask:
> *"Can you add a sentence counter to this function that reports whether the answer is 'too short' (1-2 sentences), 'good length' (3-5 sentences), or 'detailed' (6+ sentences)?"*
>
> Compare what the AI gives you to what you tried on your own. What did you get right? What did you miss?

---

## Checklist: Before You Leave

- [ ] Tested clean text on the sentiment model and saw high confidence
- [ ] Fed typos, sarcasm, mixed languages, and short text to the model
- [ ] Tried adversarial inputs (your own suggestions)
- [ ] Learned the CLEAR Framework for prompting AI to write code
- [ ] Built (or saw) the homework_analyzer function
- [ ] Used the clean_text function to fix broken inputs
- [ ] Discovered what cleaning can and cannot fix
- [ ] Browsed the Hub for text-classification models and read a model card
- [ ] Read the mystery_function and answered the questions
- [ ] Tried at least one modification challenge

**Save your work:** File > Save a copy in Drive

---

## Looking Ahead

Today you learned what makes models fail and how to fix some of those problems through data cleaning. You also learned the CLEAR Framework for getting AI to write code for you.

Next session, we go deeper into how models actually learn. You have used pre-trained models that someone else built. But what does "trained" really mean? How did those models go from knowing nothing to being useful?

Session 4 answers that question: **supervised learning** -- teaching a model by showing it labeled examples.

### Key Takeaways

1. Models can be broken by **noise**, **ambiguity**, **domain mismatch**, and **adversarial input**
2. Every model has limitations -- read the model card to find them
3. The **CLEAR Framework** helps you write better prompts for AI-assisted coding
4. **Text cleaning** fixes formatting problems but not meaning problems
5. Understanding failure is the first step toward building better systems

---

*Youth Horizons AI Researcher Program - Level 2*